<a href="https://colab.research.google.com/github/DM871/dm871.github.io/blob/master/notebooks/lab_scip_1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## The Basics: Production Allocation

The following model is for a specific instance of the production 
allocation problem seen in the first lectures. We give here the primal
and its dual model with the instantiated numerical parameters.

$$\begin{array}{*{6}{r}l}
\max & 5x_1&+&6x_2&+&8x_3&=z\\
&6x_1&+&5x_2&+&10x_3 &\leq 60\\
&8x_1&+&4x_2&+&4x_3&\leq 40\\
&4x_1&+&5x_2&+&6x_3&\leq 50\\
&x_1,&&x_2,&&x_3&\geq 0
\end{array}$$

$$\begin{array}{*{6}{r}l}
\min & 60y_1&+&40y_2&+&50y_3&=u\\
&6y_1&+&8y_2&+&4y_3 &\leq 5\\
&5y_1&+&4y_2&+&5y_3&\leq 6\\
&10y_1&+&4y_2&+&6y_3&\leq 8\\
&y_1,&&y_2,&&y_3&\geq 0
\end{array}$$

## Analysis of the final tableau 

Solving one of the two problems provides the solution also to the other problem. The primal solution is
$x^*_1=0,x^*_2=7,x^*_3=2.5$ and the dual solution is
$y^*_1=0.2,y^*_2=0,y^*_3=1$. The objective value is $z^*=u^*=62$.

We can organize these values in a *tableau*. For the given problems it looks like this:

    |------+----+----+------+----+----+----+-----|
    |   x1 | x2 | x3 |   s1 | s2 | s3 | -z |   b |
    |------+----+----+------+----+----+----+-----|
    |   ?  |  1 |  0 |  ?   |  0 |  ? |  0 |   7 |
    |   ?  |  0 |  1 |  ?   |  0 |  ? |  0 | 5/2 |
    |   ?  |  0 |  0 |  ?   |  1 |  ? |  0 |   2 |
    |------+----+----+------+----+----+----+-----|
    | -0.2 |  0 |  0 | -0.2 |  0 | -1 |  1 | -62 |
    |------+----+----+------+----+----+----+-----|

A question mark is for the values that are not relevant for the goal of
this exercise.

The three numbers of the last row in the tableau above in the columns of
the variables that are not in basis are called *reduced costs*. They
indicate how much we should make each product more expensive in order to
be worth manufacturing it. The next three values are known as *shadow
prices*. After a change of sign they give the values of the dual
variables, which are interpreted as the *marginal values* of increasing
(or decreasing) the capacities of the resources (that is, the values by
which the objective function would improve if the corresponding constraints were
relaxed by one unit, which corresponds to buying one unit more of
that resource). In our example, which seeks maximization, the marginal value
1 for the third resource means that the objective function would
increase by 1 if we had one more unit of that resource.

It can be verified that in the primal problem at the optimum the first
and third resources are fully exhausted, that is, their constraint is
satisfied at the equality, while there is *slack* for the second
resource, that is, the constraint holds with strict inequality. Looking
at the marginal values, we see that the second resource has been given a
zero valuation. This seems plausible, since we are not using all the
capacity that we have, we are not willing to place much value on it
(buying one more unit of that resource would not translate in an
improvement of the objective function).

These results are captured by the Complementary Slackness theorem of
linear programming. If a constraint is not *"binding"* in the optimal
primal solution, the corresponding dual variable is zero in the optimal
solution to the dual model. Similarly, if a constraint in the dual model
is not *"binding"* in the optimal solution to the dual model, then the
corresponding variable is zero in the optimal solution to the primal
model.


## Solving the model with PySCIPOpt

Let's write the primal model in Python and solve it with SCIP. 

In [9]:
%pip install pyscipopt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Here is the script:

In [10]:
import pyscipopt as pso

# Model
model = pso.Model("prod")

# let's use the primal simplex
model.setCharParam("lp/initalgorithm", "p")

# Create decision variables
x1 = model.addVar(name="x1", vtype="C", lb=0.0, ub=None, obj=5.0, pricedVar=False)
x2 = model.addVar("x1", "C", 0, None, 6)  # arguments by position
x3 = model.addVar(name="x3")  # arguments by deafult:  lb=0.0, ub=None, obj=0.

# The objective is to maximize (the default)
# Unecessary if we had written the obj coefficient for all vars above
model.setObjective(5.0*x1 + 6.0*x2 + 8.0*x3, "maximize")

# Add constraints to the model
model.addCons(6.0*x1 + 5.0*x2 + 10.0*x3 <= 60.0, "c1")
model.addCons(8.0*x1 + 4.0*x2 + 4.0*x3 <= 40.0, "c2")
model.addCons(4.0*x1 + 5.0*x2 + 6.0*x3 <= 50.0, "c3")

c3



The documentation for the functions `Model.addVar()` and
`Model.addConstr()`, as well as for all other functions in `pyscipopt` is
available from the [Reference
Manual](https://scipopt.github.io/PySCIPOpt/docs/html/) and more specifically from the [model API page](https://scipopt.github.io/PySCIPOpt/docs/html/pyscipopt_1_1scip_1_1Model.html)
For the variable $x_3$ the lower bound, upper bound, objective
coefficient and type are set to their default values that are,
respectively: `lb=0.0, ub=None, obj=0.0, vtype="C".
Once the model has been built, it is good practice to inspect it to make sure that it has been built as we intended.
In order to assess this, we can let PySCIPOpt write for us the set of
parameters and the model instantiated with our data. This check can be
very useful above all with implicit models as we will see later. Inspect
the files created by these lines:


In [11]:
# Write the set of SCIP parameters and their settings.
model.writeParams("param.set")
# Write the instantiated model to a file
model.writeProblem("prod1_scip.lp")  # lp format
model.writeProblem("prod1_scip.cip")  # cip format

wrote parameter settings to file /home/marco/Teaching/Courses/DM871/dm871.github.io/notebooks/param.set
wrote problem to file /home/marco/Teaching/Courses/DM871/dm871.github.io/notebooks/prod1_scip.lp
wrote problem to file /home/marco/Teaching/Courses/DM871/dm871.github.io/notebooks/prod1_scip.cip



The file `param.set` contains a list of SCIP parameters that
can be set as we did with `lp/initalgorithm`
above. Inspecting this file can provide an insight on the parameters
available as an alterntive to consult the [web page](https://scip.zib.de/doc/html/PARAMETERS.php).


The files `prod1_scip.lp` and `prod1_scip.cip`
contain the problem in two different formats.  Valid suffixes for
writing the model itself are `.mps, .rew, .lp`, or
`.rlp`.


**Your Task** Try all these formats on the production allocation
example above and check their contents. The MPS file is not very user friendly. This is
because the format is an old format when computer technology had much
more limitations than nowadays. 
The CPLEX-LP format is a more explicit version of the problem that may
be useful to check whether the model we implemented in Python is
actually the one we intended.

We are then ready to solve or optimize the model
(using `model.optimize`). You can then query the `x` attribute on the
variables to retrieve the solution (and the `name` attribute to
retrieve the variable name for each variable).

In [12]:
# Solve
model.optimize()

# Let's print the solution
if model.getStatus() == "optimal":
    print("Optimal value:", model.getObjVal())
    for v in model.getVars():
        print(v.name, " = ", model.getVal(v))
else:
    print("Problem could not be solved to optimality")



Optimal value: 62.0
x1  =  0.0
x1  =  7.0
x3  =  2.5
feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 5 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 5 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 3 variables (0 bin, 0 int, 0 impl, 3 cont) and 3 constraints
      3 constraints of type <linear>
Presolving Time: 0.00
transformed 1/1 original solutions to the transformed problem space

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts


The screen output of SCIP is described below.  If a
letter appears in front of a display row, it indicates, which heuristic
found the new primal bound, a star representing an integral
LP-relaxation.  In addition, the output indicates the amount of
presolving that is conducted. Finally, the simplex method is applied and
after 2 iterations of the primal simplex method (we set to use this
method via `model.setCharParam("lp/initalgorithm","p")`
the optimal solution is found with value 62.

```
SCIP> display display

 display column       header           position width priority status  description
 --------------       ------           -------- ----- -------- ------  -----------
 solfound                                     0     1    80000   auto  letter that indicates the heuristic which found the solution
 concsolfound                                 0     1    80000   auto  indicator that a new solution was found in concurrent solve
 time                 time                   50     5     4000   auto  total solution time
 nnodes               node                  100     7   100000   auto  number of processed nodes
 nodesleft            left                  200     7    90000   auto  number of unprocessed nodes
 lpiterations         LP iter              1000     7    30000   auto  number of simplex iterations
 lpavgiterations      LP it/n              1400     7    25000   auto  average number of LP iterations since the last output line
 concmemused          mem                  1500     5    20000   auto  total number of bytes used in block memory
 maxdepth             mdpt                 2100     5     5000   auto  maximal depth of all processed nodes
 nfrac                frac                 2500     5      700   auto  number of fractional variables in the current solution
 vars                 vars                 3000     5     3000   auto  number of variables in the problem
 conss                cons                 3100     5     3100   auto  number of globally valid constraints in the problem
 curconss             ccons                3200     5      600   auto  number of enabled constraints in current node
 curcols              cols                 3300     5      800   auto  number of LP columns in current node
 currows              rows                 3400     5      900   auto  number of LP rows in current node
 cuts                 cuts                 3500     5     2100   auto  total number of cuts applied to the LPs
 conflicts            confs                4000     5     2000   auto  total number of conflicts found in conflict analysis
 strongbranchs        strbr                5000     5     1000   auto  total number of strong branching calls
 dualbound            dualbound            9000    14    70000   auto  current global dual bound
 primalbound          primalbound         10000    14    80000   auto  current primal bound
 concprimalbound      primalbound         10000    14    80000   auto  current primal bound in concurrent solve
 gap                  gap                 20000     8    60000   auto  current (relative) gap using |primal-dual|/MIN(|dual|,|primal|)
 ```


Most of the information associated with a PySCIPOpt model is
stored in a set of attributes. Some attributes are associated with the
variables of the model, some with the constraints of the model, and some
with the model itself. To access these attributes you have to use the
methods available under Model.


### Parameter setting

It is possible to specify the parameters of the solver (for example to set a time limit) by specifying them before solving the model. For example, we can avoid preprocessing to occur and set the primal simplex as the solution method.
The list of parameters can be found in the [SCIP documentation](https://scip.zib.de/doc/html/PARAMETERS.php).

In [13]:
# Let's deactivate presolving and heuristic solutions
model.setPresolve(pso.SCIP_PARAMSETTING.OFF)
model.setHeuristics(pso.SCIP_PARAMSETTING.OFF)
model.disablePropagation()
# let's use the primal simplex
model.setCharParam("lp/initalgorithm", "p")


### The Value of the Dual and Slack variables
  The value of the dual and slack variables can be accessed by the
  methods `model.getDualsolLinear()` and
  `model.getSlack()` on the constraints. In Python:


In [14]:
# Let's print the dual variables
for c in model.getConss():
    print(c.name, model.getSlack(c), model.getDualsolLinear(c))

c1 0.0 -0.19999999999999973
c2 0.5 -0.0
c3 0.0 -1.0000000000000004


The values `c1`,`c2`,`c3` are the shadow prices (here 0.2, 0.0 and 1) which correspond to
the marginal value of the resources. The c1 and c3 constraints' value is
different from zero. This indicates that there's a variable on the upper
bound for those constraints, or in other terms that these constraints
are *"binding"*. The second constraint is not *"binding"*.

**Your Task**
Try relaxing the value of each binding constraint one at a time, solve
the modified problem, and see what happens to the optimal value of the
objective function. Also check that, as expected, changing the value of
non-binding constraints won't make any difference to the solution.

**Your Task**
We can also access several quantities associated with the variables. A
particularly relevant one is the *reduced cost*. Print the reduced costs
of the variables for our example and make sure that they correspond to
the expected values from the tableau above. \[Hint: look for the
variable attribute `rc`.\] What can we say about the solution found on
the basis of the reduced costs?

Let's now focus on the values output during the execution of the
simplex. Let's first solve another small numerical example:



Let's now focus on the values output during the execution of the
simplex. Let's first  solve another small numerical example:

In [15]:
import pyscipopt as pso

m = pso.Model("infeas")

x = m.addVar(name="x") # ie, >= 0
y = m.addVar(name="y") # ie, >= 0

m.setObjective(x - y, "maximize")

m.addCons(x + y <= 2, "c1")
m.addCons(2*x + 2*y >= 5, "c2")

m.optimize()


if m.getStatus() in ["infeasible", "unbounded"]:
    print(m.getStatus())
elif m.getStatus() == "optimal":
    print('Optimal objective: %g' % m.getObjVal())
    print( m.getVal(x) )
    print( m.getVal(y) )
    exit(0)
elif m.getStatus() != "infeasible":
    print('Optimization was stopped with status %d' % m.getStatus())
    exit(0)


infeasible
presolving:
presolving (1 rounds: 1 fast, 0 medium, 0 exhaustive):
 1 deleted vars, 1 deleted constraints, 0 added constraints, 2 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolving detected infeasibility
Presolving Time: 0.00

SCIP Status        : problem is solved [infeasible]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : -1.00000000000000e+20 (objective limit, 0 solutions)
Dual Bound         : -1.00000000000000e+20
Gap                : 0.00 %



This means that the presolve process has removed one column and
identified the model as infeasible.




**Your Task**
Resolve the production allocation example above using the primal method.
Compare the iteration values of the simplex with the previous ones. Give
your interpretation of the values Objective, Primal Inf. and Dual Inf.



In [16]:
m = pso.Model("infeas")

# Let's deactivate presolving and heuristic solutions
m.setPresolve(pso.SCIP_PARAMSETTING.OFF)
m.setHeuristics(pso.SCIP_PARAMSETTING.OFF)
m.disablePropagation()
# let's use the primal simplex
m.setCharParam("lp/initalgorithm","d")

x = m.addVar(name="x") # ie, >= 0
y = m.addVar(name="y") # ie, >= 0

m.setObjective(x - y, "maximize")

m.addCons(x + y <= 2, "c1")
m.addCons(2*x + 2*y >= 5, "c2")

m.optimize()

presolving:
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present
presolving (0 rounds: 0 fast, 0 medium, 0 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 2 variables (0 bin, 0 int, 0 impl, 2 cont) and 2 constraints
      2 constraints of type <linear>
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
  0.0s|     1 |     0 |     1 |     - |   573k |   0 |   2 |   2 |   2 |   0 |  0 |   0 |   0 |    cutoff    |      --      |   0.00%| unknown
  0.0s|     1 |     0 |     1 |     - |   573k |   0 |   2 |   2 |   2 |   0 |  0 |   0 |   0 |    cutoff    |      --      |   0.00%| unknown

SCIP Status        : problem is solved [infeasible]
Solving Tim

Try to change 5 to 2 in the right-hand-side of the second
constraint of the model above. What happens? Explain the behaviour.


#### Exporting Model Data to a File

The function `model.write(filename)` writes model data to a file. The file type
is encoded in the suffix of the file name passed to the function. Valid
suffixes for writing the model itself are `.mps, .rew, .lp,` or `.rlp`.
The suffix for writing a solution is `.sol` while `.prm` for a parameter
file.

**Your Task**

Try all these formats on the production allocation example above and
check their contents. The MPS file is not very user friendly. This is
because the format is an old format when computer technology had much
more limitations than nowadays. The CPLEX-LP format is a more explicit
version of the problem that may be useful to check whether the model we
implemented in Python is actually the one we intended.

If you have any of them installed, try solving the problem with other
solvers, eg, `cplex`, `glpk` and `scip`. For this you have to use the
MPS (Mathematical Programming System) format and run the following: 
```
    cplex -c read prod.mps optimize
    glpsol --mps prod.mps 
    scip -f prod.mps 
```
Gurobi has also a [command-line
tool](https://www.gurobi.com/documentation/9.1/refman/grb_command_line_tool.html)
to solve model files:
```
    gurobi_cl model.mps
```
You may also use the online solver at NEOS, the Network Enabled
Optimization Server supported by the US federal government and located
at Argonne National Lab. To submit an MPS model to NEOS visit
<http://www.neos-server.org/neos/>, click on the icon "NEOS Solvers",
scroll down to the Linear Programming or Mixed Integer Linear
Programming list, click on one of those, scroll down to "Model File",
click on "Choose File", select a file from your computer that contains
an MPS model, scroll down to "e-mail address:", type in your email
address, and click Submit to NEOS.

    [Standard MPS files do not indicate whether to minimize or maximize
    the objective. Thus your MPS files will come out the same whether
    the objective is minimize or maximize.

    As you are seeing, most solvers minimize the objective by default. A
    solver may have a switch to tell it to maximize instead, but that is
    different for each solver.

    If you change the signs of all the objective coefficients, while
    leaving the constraints unchanged, then minimizing the resulting MPS
    file will be equivalent to maximizing the original problem. This can
    be done easily by putting the entire objective expression in
    parentheses and placing a minus sign in front of it.]